# Debugging

Eventually, all code will fail. Even gettsim's and your code is no exception. Therefore, gettsim offers a debug mode which helps you to find the source of the error.

Let us take the same example as used in the [tutorial on the interface](interface.ipynb), but reduced to the computation of `rentev_beitr_m` which are the monthly contribution to the pension insurance.

In [1]:
import pandas as pd
from gettsim import compute_taxes_and_transfers
from gettsim import set_up_policy_environment

In the following cell, we create all necessary data to run the example.

In [2]:
data = pd.DataFrame(
    {
        "p_id": 1,
        "tu_id": 1,
        "hh_id": 1,
        "bruttolohn_m": 1000,
        "wohnort_ost": False,
    },
    index=[0],
)
data

,p_id,tu_id,hh_id,bruttolohn_m,wohnort_ost
0,1,1,1,1000,False


In [3]:
params_dict, policy_func_dict = set_up_policy_environment(2020)

In the following cell, we call the function to compute quantities in the tax and transfer system. Note that, `debug = True` which enables the debug mode. It is `False` by default. The debug mode differs from the usual call in two ways:

1. All necessary inputs to compute the `targets` and all intermediate variables are returned.
2. If an exception occurs while computing the targets, the exception is printed, but not raised. The variables which depend on the variable where the exception occurred are not computed.

The following call shows the effect of the debug mode when no exception occurs.

In [4]:
df = compute_taxes_and_transfers(
    data=data,
    user_functions=policy_func_dict,
    params=params_dict,
    targets="rentenv_beitr_m",
    debug=True,
)

The resulting DataFrame contains all necessary input variables as well as the target and intermediate variables.

In [5]:
df

,wohnort_ost,tu_id,hh_id,bruttolohn_m,p_id,_mini_job_grenze,_rentenv_beitr_bemess_grenze,_regulär_beschäftigt,_ges_beitr_arbeitsl_v_midi_jobreturn,_rentenv_beitr_regular_job,_geringfügig_beschäftigt,_in_gleitzone,_midi_job_bemessungsentgelt,_ges_beitr_rentenv_midi_job,_ag_beitr_rentenv_midi_job,_an_beitr_rentenv_midi_job,rentenv_beitr_m
0,False,1,1,1000,1,450,6900,False,NaN,NaN,False,True,961.040588,178.753549,93.0,85.753549,85.753549


## Debug mode with an exception

What happens when an exception is raised while producing the target? We artifically produce an error with a modified user function.

In [6]:
def _mini_job_grenze():
    raise ValueError

In [7]:
df = compute_taxes_and_transfers(
    data=data,
    user_functions=[policy_func_dict, _mini_job_grenze],
    params=params_dict,
    targets="rentenv_beitr_m",
    debug=True,
)

Traceback (most recent call last):
  File "xxx\gettsim\gettsim\dag.py", line 144, in execute_dag
    data[task] = dag.nodes[task]["function"](**kwargs).rename(task)
  File "<ipython-input-6-e7d88657cba8>", line 2, in _mini_job_grenze
    raise ValueError
ValueError


The traceback of the error is printed, but the execution is continued. The resulting DataFrame captures all input variables as well as variables which could be produced without errors.

In [8]:
df

,wohnort_ost,tu_id,hh_id,bruttolohn_m,p_id,_rentenv_beitr_bemess_grenze,_regulär_beschäftigt,_ges_beitr_arbeitsl_v_midi_jobreturn,_rentenv_beitr_regular_job
0,False,1,1,1000,1,6900,False,NaN,NaN
